<a href="https://colab.research.google.com/github/Cole-Krudwig/Time-Series-Predictions/blob/main/yfinance_stock_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Import Libraries
import yfinance as yf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from datetime import date

# Check if a GPU is available on Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [52]:
# Import Data
stock_symbol = input("Enter the stock ticker (e.g., AAPL): ")
today = date.today()
start_date = "2010-01-01"
end_date = today
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

Enter the stock ticker (e.g., AAPL): AAPL
[*********************100%%**********************]  1 of 1 completed


In [53]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.487534,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.498751,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.395380,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.383556,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.425995,447610800
...,...,...,...,...,...,...
2023-08-21,175.070007,176.130005,173.740005,175.839996,175.839996,46311900
2023-08-22,177.059998,177.679993,176.250000,177.229996,177.229996,42084200
2023-08-23,178.520004,181.550003,178.330002,181.119995,181.119995,52722800


In [54]:
# Extract "Close" prices and normalize the data
close_prices = stock_data["Close"].values
normalized_prices = (close_prices - np.min(close_prices)) / (np.max(close_prices) - np.min(close_prices))

# Define the sequence length
sequence_length = 10

# Create sequences
sequences = []
targets = []
for i in range(len(normalized_prices) - sequence_length):
    sequences.append(normalized_prices[i:i+sequence_length])
    targets.append(normalized_prices[i+sequence_length])

# Convert to PyTorch tensors
X = torch.tensor(sequences, dtype=torch.float32)
y = torch.tensor(targets, dtype=torch.float32)

# Reshape X to match LSTM input shape (batch_size, sequence_length, input_dim)
X = X.view(-1, sequence_length, 1)
X

tensor([[[0.0041],
         [0.0042],
         [0.0036],
         ...,
         [0.0035],
         [0.0033],
         [0.0026]],

        [[0.0042],
         [0.0036],
         [0.0035],
         ...,
         [0.0033],
         [0.0026],
         [0.0043]],

        [[0.0036],
         [0.0035],
         [0.0038],
         ...,
         [0.0026],
         [0.0043],
         [0.0037]],

        ...,

        [[0.9037],
         [0.9025],
         [0.9016],
         ...,
         [0.8842],
         [0.8913],
         [0.8986]],

        [[0.9025],
         [0.9016],
         [0.9104],
         ...,
         [0.8913],
         [0.8986],
         [0.9191]],

        [[0.9016],
         [0.9104],
         [0.8998],
         ...,
         [0.8986],
         [0.9191],
         [0.8941]]])

In [55]:
# Import DataLoader and TensorDataset
from torch.utils.data import DataLoader, TensorDataset

# Train/test split ratio
split_ratio = 0.8
split_idx = int(len(X) * split_ratio)

# Split the data
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Define batch size
batch_size = 64

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [56]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the last time step's output
        return out

# Hyperparameters
input_size = 1
hidden_size = 128
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 100

# Initialize the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Forward pass
        outputs = model(batch_X)
        batch_y = batch_y.view(-1, 1)  # Reshape batch_y to match output shape
        loss = criterion(outputs, batch_y)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        outputs = model(batch_X)
        batch_y = batch_y.view(-1, output_size)  # Reshape batch_y to match output shape
        test_loss += criterion(outputs, batch_y).item()

    avg_test_loss = test_loss / len(test_loader)
    print(f"Average Test Loss: {avg_test_loss:.6f}")

Epoch [1/100], Loss: 0.000716
Epoch [2/100], Loss: 0.000148
Epoch [3/100], Loss: 0.000147
Epoch [4/100], Loss: 0.000083
Epoch [5/100], Loss: 0.000159
Epoch [6/100], Loss: 0.000245
Epoch [7/100], Loss: 0.000093
Epoch [8/100], Loss: 0.000061
Epoch [9/100], Loss: 0.000044
Epoch [10/100], Loss: 0.000031
Epoch [11/100], Loss: 0.000043
Epoch [12/100], Loss: 0.000109
Epoch [13/100], Loss: 0.000075
Epoch [14/100], Loss: 0.000118
Epoch [15/100], Loss: 0.000069
Epoch [16/100], Loss: 0.000155
Epoch [17/100], Loss: 0.000215
Epoch [18/100], Loss: 0.000087
Epoch [19/100], Loss: 0.000046
Epoch [20/100], Loss: 0.000120
Epoch [21/100], Loss: 0.000256
Epoch [22/100], Loss: 0.000045
Epoch [23/100], Loss: 0.000046
Epoch [24/100], Loss: 0.000071
Epoch [25/100], Loss: 0.000043
Epoch [26/100], Loss: 0.000066
Epoch [27/100], Loss: 0.000066
Epoch [28/100], Loss: 0.000148
Epoch [29/100], Loss: 0.000125
Epoch [30/100], Loss: 0.000028
Epoch [31/100], Loss: 0.000089
Epoch [32/100], Loss: 0.000054
Epoch [33/100], L

In [57]:
# Create a sequence of length equal to the available data
sequence_length = len(normalized_prices)
sequence = normalized_prices[-sequence_length:]

# Convert the sequence to a PyTorch tensor
sequence_tensor = torch.tensor(sequence, dtype=torch.float32).view(1, sequence_length, 1).to(device)

# Load the trained model's state dict
model.eval()

# Use the trained model to predict tomorrow's closing price
with torch.no_grad():
    tomorrow_prediction = model(sequence_tensor)

# Inverse normalize the predicted price
tomorrow_predicted_price = (tomorrow_prediction.item() * (np.max(close_prices) - np.min(close_prices))) + np.min(close_prices)

print(f"Predicted Closing Price for Tomorrow: {tomorrow_predicted_price:.2f}")

Predicted Closing Price for Tomorrow: 175.06
